In [91]:
## Get data
import pandas as pd

fetal_health = pd.read_csv('fetal_health.csv')

fetal_health.histogram_tendency = fetal_health.histogram_tendency.astype('int').astype('str')

fetal_health = pd.get_dummies(fetal_health) ## I need to figure out how to get this to work when only passing one obs

fetal_health.fetal_health = fetal_health.fetal_health.astype('int').astype('str')

In [92]:
from sklearn.model_selection import train_test_split

fetal_health, fetal_health_test = train_test_split(fetal_health, 
                                    test_size = .2) ## withold our test set

fetal_health_train, fetal_health_validate = train_test_split(fetal_health, 
                                    test_size = .2, ) 



In [93]:
from utility_functions import rebalance_classes

fetal_health_train = rebalance_classes(fetal_health_train)

In [94]:
from sklearn.preprocessing import StandardScaler

scale = StandardScaler()

fetal_health_train_scaler = scale.fit(X = fetal_health_train.drop('fetal_health', axis = 1).to_numpy())

fetal_health_train_x = fetal_health_train_scaler.transform(\
    X = fetal_health_train.drop('fetal_health', axis = 1).to_numpy())
                                   

fetal_health_validate_x = fetal_health_train_scaler.transform(\
    X = fetal_health_validate.drop('fetal_health', axis = 1).to_numpy())


    



In [95]:
import tensorflow as tf
import numpy as np


train_y = pd.get_dummies(fetal_health_train.fetal_health).to_numpy()
validate_y = pd.get_dummies(fetal_health_validate.fetal_health).to_numpy()

train_tf_dataset = tf.data.Dataset.from_tensor_slices((fetal_health_train_x, train_y))
validate_tf_dataset = tf.data.Dataset.from_tensor_slices((fetal_health_validate_x, validate_y))


In [96]:
from tensorflow import keras
from tensorflow.keras import layers


BATCH_SIZE = 10

train_tf_dataset = train_tf_dataset.shuffle(buffer_size = fetal_health_train_x.shape[0]).batch(BATCH_SIZE)

validate_tf_dataset = validate_tf_dataset.batch(BATCH_SIZE)


In [119]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(23, input_dim=23, activation = 'relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(3, activation = 'softmax')
])


model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss='categorical_crossentropy') #use because classes are balanced

In [120]:
model.fit(train_tf_dataset, 
          validation_data = validate_tf_dataset,
          epochs=20)

Epoch 1/20
319/319 [==============================] - 5s 15ms/step - loss: 0.5906 - val_loss: 0.0000e+00
Epoch 2/20
319/319 [==============================] - 1s 4ms/step - loss: 0.3744 - val_loss: 0.3198
Epoch 3/20
319/319 [==============================] - 1s 4ms/step - loss: 0.3093 - val_loss: 0.2930
Epoch 4/20
319/319 [==============================] - 1s 4ms/step - loss: 0.2698 - val_loss: 0.2660
Epoch 5/20
319/319 [==============================] - 1s 3ms/step - loss: 0.2327 - val_loss: 0.3011
Epoch 6/20
319/319 [==============================] - 1s 3ms/step - loss: 0.2153 - val_loss: 0.2527
Epoch 7/20
319/319 [==============================] - 1s 4ms/step - loss: 0.2008 - val_loss: 0.2788
Epoch 8/20
319/319 [==============================] - 1s 4ms/step - loss: 0.1810 - val_loss: 0.2495
Epoch 9/20
319/319 [==============================] - 1s 4ms/step - loss: 0.1685 - val_loss: 0.2706
Epoch 10/20
319/319 [==============================] - 1s 4ms/step - loss: 0.1586 - val_loss: 0

In [121]:
raw_preds = model.predict(validate_tf_dataset)

In [122]:
preds = np.apply_along_axis(np.argmax, 1, raw_preds) + 1

preds = preds.transpose()

preds = pd.DataFrame(preds, columns = ['preds'], dtype = 'str')



In [123]:
from utility_functions import accuracy_by_class
from utility_functions import print_metrics


if 'level_0' in fetal_health_validate.columns: ## The reset_index function 
    ## creates this column and creates an error if it's already there
    fetal_health_validate.drop('level_0', axis = 1, inplace = True)

fetal_health_validate = fetal_health_validate.reset_index() 



print(print_metrics(accuracy_by_class(preds.preds.astype('str'), fetal_health_validate.fetal_health.astype('str'))))



Overall accuracy: 0.9088235294117647
Accuracy for class 1: 0.9266409266409267
Accuracy for class 2: 0.8771929824561403
Accuracy for class 3: 0.7916666666666666
Weighted Accuracy: 0.8651668585879112
Macro F1 score: 0.8492328927111537



In [112]:
#TODO: reduce overfitting by:
# 1. Data augmentation instead of just oversampling



SyntaxError: invalid syntax (<ipython-input-112-ecf84786f158>, line 1)